# 🌩️ **CloudLeecher: Production Backend**

Welcome to the **CloudLeecher** backend. This notebook turns your Google Colab instance into a powerful, high-speed torrent downloader that saves files directly to your Google Drive.

### **Instructions**
1.  **Mount Drive**: Connect your Google storage.
2.  **Install**: Clone repo and set up the environment.
3.  **Start Services**: Launch the backend services.
4.  **Connect**: Paste the URL and API Key into the CloudLeecher Frontend.

## 1. 📂 **Mount Google Drive**
We need access to your Drive to save the downloaded files.

In [ ]:
from google.colab import drive
import os

# Mount Google Drive
drive.mount('/content/drive')

# Define and Create Download Directory
DOWNLOAD_DIR = "/content/drive/MyDrive/TorrentDownloads"
os.makedirs(DOWNLOAD_DIR, exist_ok=True)

print(f"✅ Download Directory Ready: {DOWNLOAD_DIR}")

## 2. 🛠️ **Install Dependencies**
Cloning the CloudLeecher repository and installing dependencies.

In [ ]:
%%capture
!apt-get update -qq
!apt-get install -y -qq aria2

import os
# Clone or Pull Repo
if not os.path.exists('/content/CloudLeecher'):
    !git clone https://github.com/heavens7above/CloudLeecher.git
else:
    !cd CloudLeecher && git pull

!pip install -r CloudLeecher/backend/requirements.txt

print("✅ All dependencies installed successfully.")

## 3. 🚀 **Start Downloader Service**
Initializing the Aria2 RPC server in the background (using local temp storage for reliability).

In [ ]:
import subprocess
import os

# Use local temp dir for reliability (FUSE workaround)
TEMP_DIR = "/content/temp_downloads"
os.makedirs(TEMP_DIR, exist_ok=True)

# Start Aria2c as a daemon process
cmd = [
    "aria2c",
    "--enable-rpc",
    "--rpc-listen-all=true",
    "--rpc-allow-origin-all",
    f"--dir={TEMP_DIR}",
    "--file-allocation=none",
    "--max-connection-per-server=16",
    "--split=16",
    "--min-split-size=1M",
    "--seed-time=0",
    "--daemon=true"
]

subprocess.run(
    cmd,
    stdout=subprocess.DEVNULL,
    stderr=subprocess.DEVNULL
)

print(f"✅ Aria2 Background Service Started (Temp Dir: {TEMP_DIR}).")

## 4. 🌐 **Launch Secure Backend**
Starting the application securely with an auto-generated API Key.

> **⚠️ Important**: Ensure you have added your Ngrok Authtoken to Colab Secrets with the key `NGROK-AUTHTOKEN`.

In [ ]:
from pyngrok import ngrok
from google.colab import userdata
import subprocess
import sys
import time
import os
import uuid

# 1. Generate Security Key
api_key = str(uuid.uuid4())
os.environ['CLOUDLEECHER_API_KEY'] = api_key
os.environ['TEMP_DOWNLOAD_DIR'] = "/content/temp_downloads"
# DRIVE_MOUNT_PATH defaults to /content/drive in app.py

# 2. Authenticate Ngrok
try:
    AUTH_TOKEN = userdata.get("NGROK-AUTHTOKEN")
    ngrok.set_auth_token(AUTH_TOKEN)
except Exception as e:
    print("❌ Error: Ngrok Auth Token not found! Please add 'NGROK-AUTHTOKEN' to Colab Secrets.")
    raise e

# 3. Cleanup Old Processes
ngrok.kill()
os.system("fuser -k 5000/tcp > /dev/null 2>&1")

# 4. Start Flask App in Background
log_file = open("/content/flask.log", "w")
subprocess.Popen(
    [sys.executable, "/content/CloudLeecher/backend/app.py"], 
    stdout=log_file, 
    stderr=log_file,
    env=os.environ.copy()
)
time.sleep(3)  # Allow Flask to initialize

# 5. Open Ngrok Tunnel
try:
    public_url = ngrok.connect(5000).public_url
    print("\n" + "="*60)
    print(f"🔗 PUBLIC URL: {public_url}")
    print(f"🔑 API KEY:    {api_key}")
    print("="*60 + "\n")
    print("✅ CloudLeecher Backend is Online!")
    print("🌍 Frontend App: https://cloudleecher.web.app")
    print("📋 Copy the URL and API KEY above into the CloudLeecher Frontend app.")

    # Keep cell running to keep thread alive
    while True:
        time.sleep(10)
except Exception as e:
    print(f"❌ Failed to start Ngrok: {e}")